In [1]:
from datasets import load_dataset
import pandas as pd

/home/ubuntu/miniconda3/envs/rapids_singlecell/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
drug_metadata = pd.DataFrame(load_dataset("tahoebio/Tahoe-100M","drug_metadata", split="train"))
drug_metadata = drug_metadata.loc[drug_metadata["targets"].notna()]
target_to_drug = dict(zip(drug_metadata["targets"], drug_metadata["drug"]))
cell_metadata = pd.DataFrame(load_dataset("tahoebio/Tahoe-100M","cell_line_metadata", split="train"))
depmap = pd.read_csv("../CRISPRGeneDependency.csv", index_col=0)

In [4]:
depmap_to_tahoe = dict(zip(cell_metadata["Cell_ID_DepMap"], cell_metadata["Cell_ID_Cellosaur"]))
depmap.index = depmap.index.map(depmap_to_tahoe)
depmap = depmap.loc[depmap.index.notna()]
depmap.columns = depmap.columns.map(lambda x: x.split(" ")[0])

In [5]:
depmap = depmap.reset_index().melt(id_vars='index')
depmap["drug"] = depmap["variable"].map(target_to_drug)

In [6]:
depmap = depmap.dropna(subset=["drug"])
depmap["drug_cellline"] = depmap["drug"] + "_" + depmap["index"]

In [9]:
depmap.index = depmap["drug_cellline"]
depmap["depmap_dependency_score"] = depmap["value"]
depmap[["depmap_dependency_score"]].to_csv("data_for_classifier/depmap_dependency_scores.tsv", index=True, sep='\t')